In [1]:
import os
os.environ['KMP_DUPLICATE_LIB_OK'] = 'TRUE'
import sys
import torch as t
from torch import nn, Tensor
from torch.distributions.categorical import Categorical
from torch.nn import functional as F
from dataclasses import dataclass
import numpy as np
import einops
from jaxtyping import Float, Int
from typing import Optional, Callable, Union, List, Tuple, Dict
from functools import partial
from tqdm.notebook import tqdm
from dataclasses import dataclass
from rich import print as rprint
from rich.table import Table
from IPython.display import display, HTML
from pathlib import Path
from scipy.optimize import fsolve

# Make sure exercises are in the path
sys.path.append(str(Path.cwd().parent))



device = t.device("cuda" if t.cuda.is_available() else "cpu")

sys.path.append('../Models_of_superposition/')
from superposition_lib import Config, get_angle, create_mask_from_group_members, create_diagonal_mask,Model

from sae_lib import AutoEncoder, AutoEncoderConfig




C:\Users\wusch\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\scipy\__init__.py:169: UserWarning: A NumPy version >=1.18.5 and <1.26.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:

dicts_of_models = t.load("../Models_of_superposition/data/experiment_1/model.pth")
feature_probability = t.load("../Models_of_superposition/data/experiment_1/feature_probability.pth")
group_size = 5
inner_goup_distance = 0.1
p_transfer = 0.9
n_instances = len(feature_probability)
n_features = dicts_of_models["W"].shape[2]
n_hidden = dicts_of_models["W"].shape[1]
groupings =[[dict(members=list(range(i, i+group_size)), semantic_distance=inner_goup_distance, p_transfer = p_transfer) for i in range(0, n_features, group_size)] for _ in range(n_instances)]
cfg = Config(
    n_instances = n_instances,
    n_features = n_features,
    n_hidden = n_hidden,
)

model = Model(
    cfg = cfg,
    device = device,
    feature_probability = feature_probability,
    groupings = groupings,
)
model.load_state_dict(dicts_of_models)

<All keys matched successfully>

In [3]:
model.feature_probability = t.ones_like(model.feature_probability)*.1
model = model.to(device)


In [4]:
SAE_config = AutoEncoderConfig(n_instances,20,80,l1_coeff = 0.1)
SAE = AutoEncoder(SAE_config)

In [5]:
data_log, loss_log = SAE.optimize(model, steps = 1500)

  0%|          | 0/1500 [00:00<?, ?it/s]

In [6]:
data_path = Path("data/sae_3")
data_path.mkdir(exist_ok=True)
SAE = SAE.to("cpu")
t.save(SAE.state_dict(), data_path / "SAE_model.pth")

In [7]:
t.save(data_log, data_path / "SAE_model_data_log.pth")
t.save(loss_log, data_path / "SAE_model_loss_log.pth")

In [6]:
data_path = Path("data")
state_dict = t.load(data_path / "SAE_model.pth")

In [9]:
SAE.load_state_dict(state_dict)

<All keys matched successfully>

In [10]:
SAE.to("cpu")

AutoEncoder()